In [ ]:
import datetime

**Prerequisites**

In [ ]:
!pip3 install dimcli plotly networkx pyvis jsonpickle  -U --quiet

In [ ]:
!pip3 install dimcli

In [ ]:
import dimcli
from dimcli.utils import *
from dimcli.utils.networkviz import NetworkViz # custom version of pyvis - colab-compatible

import json
import sys
import pandas as pd
import networkx as nx
import plotly.express as px
import itertools

In [ ]:
print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
  import getpass
  KEY = getpass.getpass(prompt='API Key: ')
  dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
  KEY = ""
  dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

In [ ]:
dsl = dimcli.Dsl()

In [ ]:
abstract.count('films')

In [ ]:
1/27

In [ ]:
dimcli.extract_concepts("text of abstract")

In [ ]:
len(abstract.split())

In [ ]:
abstract = """We describe monocrystalline graphitic films, which are a few atoms thick but are nonetheless stable under ambient conditions,
metallic, and of remarkably high quality. The films are found to be a two-dimensional semimetal with a tiny overlap between
valence and conductance bands, and they exhibit a strong ambipolar electric field effect such that electrons and
holes in concentrations up to 10 per square centimeter and with room-temperature mobilities of approximately 10,000 square
centimeters per volt-second can be induced by applying gate voltage.
"""
res = dsl.query(f"""extract_concepts("{abstract}", return_scores=true)""")
pd.DataFrame(res['extracted_concepts'])



In [1]:
import pandas as pd
import os
from collections import Counter
import networkx as nx
import itertools
os.getcwd()

'/Users/zhuchen/TokyoU/UTokyo PhD/UTokyo_2023_Spring/DLT_companies'

In [158]:
data_APAC

Index(['Organization Name', 'Industries', 'Description', 'Founded Date',
       'Headquarters Location', 'Headquarters Regions'],
      dtype='object')

In [159]:
data_US = pd.read_csv('US_DLT.csv')
data_EU = pd.read_csv('EU_DLT.csv')
data_APAC = pd.read_csv('APAC_DLT.csv')
data_APAC.columns


Index(['Organization Name', 'Organization Name URL', 'Number of Employees',
       'Industries', 'Last Funding Type', 'Contact Job Departments',
       'IPqwery - Patents Granted', 'Number of Investments', 'IPO Status',
       'IPqwery - Total Patents', 'Closed Date', 'Closed Date Precision',
       'Headquarters Location', 'Headquarters Regions',
       'Diversity Spotlight (US Headquarters Only)', 'Estimated Revenue Range',
       'Description', 'Founded Date', 'Founded Date Precision',
       'Operating Status', 'Exit Date', 'Exit Date Precision', 'Company Type',
       'Website', 'Twitter', 'Facebook', 'LinkedIn', 'Contact Email',
       'Phone Number', 'Hub Tags', 'Number of Articles', 'Full Description',
       'Actively Hiring', 'Announced Date', 'Announced Date Precision'],
      dtype='object')

In [166]:
list(data_US[data_US['Organization Name'] == 'MovoCash']['Industries'])

['Blockchain, Cryptocurrency, Financial Services, FinTech, Mobile Payments, Payments']

In [169]:
x = ['Blockchain', 'Cryptocurrency', 'Financial Services', 'FinTech', 'Mobile Payments', 'Payments']
len(list(itertools.combinations(x, 2)))

15

In [182]:
data = pd.read_csv('APAC_DLT.csv')
data = data[['Organization Name', 'Industries', 'Description', 'Founded Date', 'Headquarters Location', 'Headquarters Regions']]
data['Founded Year'] = [int(i[:4]) for i in list(data['Founded Date'])]

#2008 2013 2018 2023
#focal_year = 2015
#data = data[data['Founded Year'] <= focal_year]

Industries = list(data['Industries'])
Industries = sum([Industry.split(',') for Industry in Industries], [])
Industries = [i.strip() for i in Industries]
Industries_set = list(set(Industries))
Industries_count = Counter(Industries)
print('The total number of industries:', len(Industries_set), end='.')

The total number of industries: 405.

In [183]:
Industries = list(data['Industries'])
Industries = [Industry.split(',') for Industry in Industries]
edges = []
for industries in Industries:
    industries1 = [i.strip() for i in industries]
    sub_edges = list(itertools.combinations(industries1, 2))
    edges.extend(sub_edges)

edge_set = list(set(edges))
edge_count = Counter(edges)

In [184]:
G = nx.Graph() # networkX instance

MIN_EDGE_WEIGHT = 30

for industry in Industries_set:
    G.add_node(industry, frequency=Industries_count[industry])
print("Nodes:", len(G.nodes()), "Edges:", len(G.edges()))

for edge in edge_set:
    G.add_edge(edge[0], edge[1], weight = edge_count[edge])
    
print(f".. cleaning up edges with weight < {MIN_EDGE_WEIGHT}...")

for a, b, w in list(G.edges(data='weight')):
    if w < MIN_EDGE_WEIGHT:
        G.remove_edge(a, b)
print("Nodes:", len(G.nodes()), "Edges:", len(G.edges()))

print(f".. removing isolated nodes...")

G.remove_nodes_from(list(nx.isolates(G)))
print("Nodes:", len(G.nodes()), "Edges:", len(G.edges()))

Nodes: 405 Edges: 0
.. cleaning up edges with weight < 30...
Nodes: 405 Edges: 85
.. removing isolated nodes...
Nodes: 40 Edges: 85


In [185]:
from pyvis.network import Network
import plotly.express as px

viznet = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", filter_menu=True)
#viznet.toggle_hide_edges_on_drag(True)
viznet.barnes_hut()
#viznet.repulsion(300)
#viznet.heading = f"Concepts co-occurrence for '{KEYWORD}' publications"


# reuse plotly color palette
palette = px.colors.diverging.Temps  # 7 colors

viznet.from_nx(G)


# update visual features

for node in viznet.nodes:
    freq = G.nodes[node['label']]['frequency']
    #score_avg = G.nodes[node['label']]['score_avg']
    #score_bucket = G.nodes[node['label']]['score_bucket'] # get from original network

    node['size'] = freq * 0.1
    #node['color'] = palette[3*score_bucket]  # get color based on score_bucket (1 or 2)
    node['borderWidthSelected'] = 5
    #node['title'] = f"<h4>Concept: '{node['label']}'</h4><hr>Frequency: {freq}",
    
    node['title'] = f"Concept: '{node['label']}', Frequency: {freq}"
    # print(node)

G = nx.Graph() # networkX instance

#MIN_EDGE_WEIGHT = 100

for industry in Industries_set:
    G.add_node(industry, frequency=Industries_count[industry])
print("Nodes:", len(G.nodes()), "Edges:", len(G.edges()))

for edge in edge_set:
    G.add_edge(edge[0], edge[1], weight = edge_count[edge])
    
print(f".. cleaning up edges with weight < {MIN_EDGE_WEIGHT}...")

for a, b, w in list(G.edges(data='weight')):
    if w < MIN_EDGE_WEIGHT:
        G.remove_edge(a, b)
print("Nodes:", len(G.nodes()), "Edges:", len(G.edges()))

print(f".. removing isolated nodes...")

G.remove_nodes_from(list(nx.isolates(G)))
print("Nodes:", len(G.nodes()), "Edges:", len(G.edges()))    
    

for edge in viznet.edges:
    # get value from main Network weight
    edge['value'] = G.edges[edge['from'], edge['to']]['weight'] * 10
    # print(edge)

viznet.show("concepts_network_APAC.html")

Nodes: 405 Edges: 0
.. cleaning up edges with weight < 30...
Nodes: 405 Edges: 85
.. removing isolated nodes...
Nodes: 40 Edges: 85
